# First-Order Logic (FOL) Syntax and Semantics

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand First-Order Logic (FOL) syntax and semantics
- Work with predicates, quantifiers (∀, ∃), and variables
- Write code to parse and evaluate FOL formulas
- Apply logical reasoning with FOL to solve AI problems
- Understand the difference between propositional logic and FOL

## 🔗 Prerequisites

- ✅ Completed: Propositional Logic and Truth Tables notebooks
- ✅ Understanding of logical operators and inference rules

---

## Official Structure Reference

This notebook covers practical activities from **Course 02, Unit 2**:
- Working with First-Order Logic (FOL) syntax and semantics
- Writing code to parse and evaluate FOL formulas
- Applying logical reasoning to solve AI problems (like knowledge graph reasoning)
- **Source:** `DETAILED_UNIT_DESCRIPTIONS.md` - Unit 2 Practical Content

---

## Introduction to First-Order Logic

**First-Order Logic (FOL)** extends propositional logic by adding:
- **Predicates**: Properties of objects (e.g., Human(x), Mortal(x))
- **Quantifiers**: Universal (∀) and Existential (∃)
- **Variables**: Represent objects (x, y, z)
- **Functions**: Operations on objects

**Key Difference from Propositional Logic:**
- Propositional Logic: Works with simple propositions (True/False)
- FOL: Can express relationships and properties of objects


In [1]:
import re
from typing import Dict, List, Set, Callable, Any

print("✅ Libraries imported!")
print("Ready to work with First-Order Logic!")


✅ Libraries imported!
Ready to work with First-Order Logic!


## Part 1: FOL Syntax - Predicates and Terms

Let's start by representing FOL predicates and terms in Python.


In [2]:
class FOLPredicate:
    """Represents a FOL predicate (e.g., Human(x), Loves(x, y))"""
    
    def __init__(self, name: str, arguments: List[str]):
        self.name = name
        self.arguments = arguments  # List of variables or constants
    
    def __str__(self):
        args_str = ', '.join(self.arguments)
        return f"{self.name}({args_str})"
    
    def __repr__(self):
        return str(self)
    
    def evaluate(self, interpretation: Dict[str, Any]) -> bool:
        """
        Evaluate predicate with given interpretation (variable assignments).
        In a real system, this would check against a knowledge base.
        """
        # For demonstration: check if all arguments are in interpretation
        # In practice, this would query a knowledge base
        for arg in self.arguments:
            if arg not in interpretation:
                return False  # Cannot evaluate if variable not bound
        return True  # Simplified: assume true if all variables bound

# Example: Create predicates
print("=" * 60)
print("FOL Predicate Examples:")
print("=" * 60)

# Human(socrates)
human_pred = FOLPredicate("Human", ["socrates"])
print(f"1. {human_pred}")

# Loves(romeo, juliet)
loves_pred = FOLPredicate("Loves", ["romeo", "juliet"])
print(f"2. {loves_pred}")

# Mortal(x) - with variable
mortal_pred = FOLPredicate("Mortal", ["x"])
print(f"3. {mortal_pred}")

# Greater(x, y) - binary predicate
greater_pred = FOLPredicate("Greater", ["x", "y"])
print(f"4. {greater_pred}")


FOL Predicate Examples:
1. Human(socrates)
2. Loves(romeo, juliet)
3. Mortal(x)
4. Greater(x, y)


## Part 2: Quantifiers (Universal ∀ and Existential ∃)

Quantifiers allow us to express statements about all objects or some objects.


In [3]:
class FOLFormula:
    """Represents a FOL formula with quantifiers"""
    
    def __init__(self, formula_type: str, variable: str = None, 
                 predicate: FOLPredicate = None, subformula: 'FOLFormula' = None):
        """
        formula_type: 'quantifier' (∀, ∃), 'predicate', 'not', 'and', 'or', 'implies'
        """
        self.formula_type = formula_type
        self.variable = variable
        self.predicate = predicate
        self.subformula = subformula
    
    def __str__(self):
        if self.formula_type == 'universal':
            return f"∀{self.variable}. {self.subformula}"
        elif self.formula_type == 'existential':
            return f"∃{self.variable}. {self.subformula}"
        elif self.formula_type == 'predicate':
            return str(self.predicate)
        else:
            return f"{self.formula_type}({self.subformula})"

# Example 1: Universal Quantifier - All humans are mortal
# ∀x. Human(x) → Mortal(x)
print("=" * 60)
print("Universal Quantifier Examples:")
print("=" * 60)

human_x = FOLPredicate("Human", ["x"])
mortal_x = FOLPredicate("Mortal", ["x"])

# For simplicity, we'll represent the implication
all_humans_mortal = FOLFormula('universal', 'x', 
                               subformula=FOLFormula('implies', 
                                                    predicate=human_x,
                                                    subformula=FOLFormula('predicate', predicate=mortal_x)))
print(f"∀x. Human(x) → Mortal(x)")
print("Meaning: For all x, if x is Human, then x is Mortal")

# Example 2: Existential Quantifier - Someone loves Romeo
# ∃x. Loves(x, romeo)
print("\n" + "=" * 60)
print("Existential Quantifier Examples:")
print("=" * 60)

loves_x_romeo = FOLPredicate("Loves", ["x", "romeo"])
someone_loves_romeo = FOLFormula('existential', 'x', 
                                 predicate=loves_x_romeo)
print(f"∃x. Loves(x, romeo)")
print("Meaning: There exists an x such that x loves romeo")


Universal Quantifier Examples:
∀x. Human(x) → Mortal(x)
Meaning: For all x, if x is Human, then x is Mortal

Existential Quantifier Examples:
∃x. Loves(x, romeo)
Meaning: There exists an x such that x loves romeo


## Part 3: Simple FOL Formula Parser

Let's create a simple parser to convert FOL formulas from string representation to our data structure.


In [4]:
class FOLParser:
    """Simple parser for FOL formulas"""
    
    @staticmethod
    def parse_predicate(pred_str: str) -> FOLPredicate:
        """Parse a predicate string like 'Human(x)' or 'Loves(x, y)'"""
        # Match pattern: Name(arg1, arg2, ...)
        match = re.match(r'(\w+)\(([^)]+)\)', pred_str.strip())
        if not match:
            raise ValueError(f"Invalid predicate format: {pred_str}")
        
        name = match.group(1)
        args_str = match.group(2)
        arguments = [arg.strip() for arg in args_str.split(',')]
        
        return FOLPredicate(name, arguments)
    
    @staticmethod
    def parse_universal(formula_str: str) -> FOLFormula:
        """Parse universal quantifier: ∀x. P(x)"""
        # Match: ∀x. formula
        match = re.match(r'∀(\w+)\.\s*(.+)', formula_str.strip())
        if not match:
            raise ValueError(f"Invalid universal quantifier: {formula_str}")
        
        variable = match.group(1)
        subformula_str = match.group(2).strip()
        
        # For simplicity, parse the subformula as a predicate
        predicate = FOLParser.parse_predicate(subformula_str)
        subformula = FOLFormula('predicate', predicate=predicate)
        
        return FOLFormula('universal', variable, subformula=subformula)

# Test parser
print("=" * 60)
print("FOL Formula Parser:")
print("=" * 60)

# Parse predicates
pred1 = FOLParser.parse_predicate("Human(socrates)")
print(f"Parsed: {pred1}")

pred2 = FOLParser.parse_predicate("Loves(romeo, juliet)")
print(f"Parsed: {pred2}")

# Parse universal quantifier
universal = FOLParser.parse_universal("∀x. Human(x)")
print(f"Parsed: {universal}")


FOL Formula Parser:
Parsed: Human(socrates)
Parsed: Loves(romeo, juliet)
Parsed: ∀x. Human(x)


## Part 4: Evaluating FOL Formulas

Now let's create a simple evaluator that can check if FOL formulas are true given a knowledge base.


In [5]:
class KnowledgeBase:
    """Simple knowledge base for storing facts"""
    
    def __init__(self):
        self.facts = []  # List of (predicate_name, arguments_tuple) -> bool
    
    def add_fact(self, predicate: FOLPredicate, value: bool = True):
        """Add a fact to the knowledge base"""
        self.facts.append((predicate.name, tuple(predicate.arguments), value))
    
    def check(self, predicate: FOLPredicate) -> bool:
        """Check if a predicate is true in the knowledge base"""
        fact_key = (predicate.name, tuple(predicate.arguments))
        for fact_name, fact_args, fact_value in self.facts:
            if fact_name == predicate.name and fact_args == tuple(predicate.arguments):
                return fact_value
        return False  # Closed world assumption: not found = false
    
    def get_all(self, predicate_name: str, variable_positions: List[int]):
        """Get all facts matching a pattern (for quantifier evaluation)"""
        results = []
        for fact_name, fact_args, fact_value in self.facts:
            if fact_name == predicate_name:
                results.append((fact_args, fact_value))
        return results

# Example: Socrates is mortal argument in FOL
print("=" * 60)
print("FOL Reasoning Example: Socrates is Mortal")
print("=" * 60)

# Create knowledge base
kb = KnowledgeBase()

# Add facts
kb.add_fact(FOLPredicate("Human", ["socrates"]), True)
kb.add_fact(FOLPredicate("Human", ["plato"]), True)

# Add rule: All humans are mortal
# This means: For any x, if Human(x) is true, then Mortal(x) is true

# Check if Socrates is human
socrates_human = FOLPredicate("Human", ["socrates"])
if kb.check(socrates_human):
    print("✅ Socrates is Human (fact in KB)")
    
    # Since all humans are mortal (rule), and Socrates is human, then:
    # Socrates is mortal
    print("✅ All humans are mortal (rule)")
    print("✅ Therefore: Socrates is mortal (inferred)")
    
    # Add the inferred fact
    kb.add_fact(FOLPredicate("Mortal", ["socrates"]), True)
    print(f"✅ Added to KB: Mortal(socrates)")


FOL Reasoning Example: Socrates is Mortal
✅ Socrates is Human (fact in KB)
✅ All humans are mortal (rule)
✅ Therefore: Socrates is mortal (inferred)
✅ Added to KB: Mortal(socrates)


## Part 5: Applying FOL to AI Reasoning Problems

Let's apply FOL to a practical AI reasoning problem.


In [6]:
# Example: Knowledge Graph Reasoning with FOL
# We'll represent relationships and infer new knowledge

print("=" * 60)
print("AI Reasoning Problem: Family Relationships")
print("=" * 60)

family_kb = KnowledgeBase()

# Add facts about family relationships
family_kb.add_fact(FOLPredicate("Parent", ["alice", "bob"]), True)
family_kb.add_fact(FOLPredicate("Parent", ["bob", "charlie"]), True)

# Rule: Grandparent relation
# ∀x, y, z. Parent(x, y) ∧ Parent(y, z) → Grandparent(x, z)

# Check: Is Alice a grandparent of Charlie?
alice_parent_bob = FOLPredicate("Parent", ["alice", "bob"])
bob_parent_charlie = FOLPredicate("Parent", ["bob", "charlie"])

if family_kb.check(alice_parent_bob) and family_kb.check(bob_parent_charlie):
    print("✅ Parent(alice, bob) is True")
    print("✅ Parent(bob, charlie) is True")
    print("✅ Applying rule: Parent(x, y) ∧ Parent(y, z) → Grandparent(x, z)")
    print("✅ Therefore: Grandparent(alice, charlie)")
    
    # Add inferred fact
    family_kb.add_fact(FOLPredicate("Grandparent", ["alice", "charlie"]), True)
    print("✅ Added to KB: Grandparent(alice, charlie)")


AI Reasoning Problem: Family Relationships
✅ Parent(alice, bob) is True
✅ Parent(bob, charlie) is True
✅ Applying rule: Parent(x, y) ∧ Parent(y, z) → Grandparent(x, z)
✅ Therefore: Grandparent(alice, charlie)
✅ Added to KB: Grandparent(alice, charlie)


## Summary

### Key Concepts Learned:

1. **First-Order Logic (FOL)**
   - Extends propositional logic with predicates, quantifiers, and variables
   - Can express relationships and properties of objects
   - More expressive than propositional logic

2. **FOL Components**
   - **Predicates**: Human(x), Loves(x, y)
   - **Quantifiers**: Universal (∀) and Existential (∃)
   - **Variables**: x, y, z represent objects
   - **Functions**: Operations on objects

3. **FOL Syntax**
   - Universal: ∀x. P(x) - "For all x, P(x)"
   - Existential: ∃x. P(x) - "There exists x such that P(x)"

4. **FOL Evaluation**
   - Requires a knowledge base (set of facts)
   - Can infer new facts using rules
   - Enables automated reasoning

### Real-World Applications:
- Expert systems (medical diagnosis, legal reasoning)
- Knowledge representation and reasoning
- Database query languages (SQL uses FOL concepts)
- Automated theorem proving
- Natural language understanding

### Difference from Propositional Logic:
- **Propositional Logic**: "It is raining" (simple True/False)
- **FOL**: "For all humans x, x is mortal" (can express general rules)

**Reference:** This notebook covers Course 02, Unit 2 requirements: "Working with First-Order Logic (FOL) syntax and semantics" and "Writing code to parse and evaluate FOL formulas"
